<a href="https://colab.research.google.com/github/Lshmyrko/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [50.4 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,346 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,074 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,796 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [5]:
# 2. Create a temporary view of the DataFrame.
import urllib.request

# Download the CSV file locally
local_file_path = "/tmp/home_sales_revised.csv"
urllib.request.urlretrieve(url, local_file_path)

# Read the locally downloaded CSV file into a DataFrame
df = spark.read.option("header", "true").csv("file://" + local_file_path)

# Show the first few rows of the DataFrame
df.show(5)

# Create a temporary view of the DataFrame
df.createOrReplaceTempView("home_sales_view")




+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [6]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
# Display the schema of the DataFrame
df.printSchema()

# Perform a SQL query to calculate the average price for a four-bedroom house per year
query = """
SELECT
    SUBSTRING(date, 1, 4) AS year,
    AVG(CAST(price AS DOUBLE)) AS average_price
FROM
    home_sales_view
WHERE
    bedrooms = 4
GROUP BY
    SUBSTRING(date, 1, 4)
ORDER BY
    year
"""

# Run the query and show the result
result = spark.sql(query)
result.show()


root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)

+----+------------------+
|year|     average_price|
+----+------------------+
|2019| 300263.6955128205|
|2020|298353.78003169573|
|2021|   301819.44398864|
|2022| 296363.8845050215|
+----+------------------+



In [7]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
SELECT
    date_built AS year_built,
    AVG(CAST(price AS DOUBLE)) AS average_price
FROM
    home_sales_view
WHERE
    bedrooms = 3
    AND bathrooms = 3
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Run the query and show the result
result = spark.sql(query)
result.show()


+----------+------------------+
|year_built|     average_price|
+----------+------------------+
|      2010|  292859.615942029|
|      2011|291117.46706586826|
|      2012| 293683.1872074883|
|      2013|295962.27145085804|
|      2014| 290852.2661870504|
|      2015| 288770.2966101695|
|      2016|  290555.073964497|
|      2017| 292676.7887740029|
+----------+------------------+



In [8]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
SELECT
    date_built AS year_built,
    AVG(CAST(price AS DOUBLE)) AS average_price
FROM
    home_sales_view
WHERE
    bedrooms = 3
    AND bathrooms = 3
    AND floors = 2
    AND sqft_living >= 2000
GROUP BY
    date_built
ORDER BY
    date_built
"""

# Run the query and show the result
result = spark.sql(query)
result.show()


+----------+------------------+
|year_built|     average_price|
+----------+------------------+
|      2010| 285010.2215909091|
|      2011| 276553.8128654971|
|      2012|307539.97402597405|
|      2013|      303676.79375|
|      2014| 298264.7183908046|
|      2015| 297609.9679144385|
|      2016| 293965.1046511628|
|      2017|280317.57692307694|
+----------+------------------+



In [9]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
# Perform a SQL query to calculate the average price per "view" rating
query = """
SELECT
    view,
    AVG(CAST(price AS DOUBLE)) AS average_price
FROM
    home_sales_view
GROUP BY
    view
HAVING
    AVG(CAST(price AS DOUBLE)) >= 350000
ORDER BY
    view DESC
"""

# Start the timer
start_time = time.time()

# Run the query and show the result
result = spark.sql(query)
result.show()

# Stop the timer and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))




+----+------------------+
|view|     average_price|
+----+------------------+
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
|  80| 991767.3783783783|
+----+------------------+
only showing top 20 rows

--- 0.875967264175415 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales_view")

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales_view')

True

In [16]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

# Run the query using the cached data
result_cached = spark.sql(query)
result_cached.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+
|view|     average_price|
+----+------------------+
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
|  80| 991767.3783783783|
+----+------------------+
only showing top 20 rows

--- 2.369760274887085 seconds ---


In [19]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
parquet_path = "/tmp/home_sales_partitioned.parquet"
df.write.partitionBy("date_built").parquet(parquet_path)

In [20]:
# 11. Read the parquet formatted data.
partitioned_df = spark.read.parquet(parquet_path)
partitioned_df.show(5)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [21]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("home_sales_partitioned_view")


In [23]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.


# Define the SQL query
query_partitioned = """
SELECT
    view,
    AVG(CAST(price AS DOUBLE)) AS average_price
FROM
    home_sales_partitioned_view
GROUP BY
    view
HAVING
    AVG(CAST(price AS DOUBLE)) >= 350000
ORDER BY
    view DESC
"""

# Start the timer
start_time_partitioned = time.time()

# Run the query on the partitioned data and show the result
result_partitioned = spark.sql(query_partitioned)
result_partitioned.show()

# Stop the timer and print the runtime
print("--- %s seconds ---" % (time.time() - start_time_partitioned))


+----+------------------+
|view|     average_price|
+----+------------------+
|  99|1061201.4193548388|
|  98|1053739.3333333333|
|  97| 1129040.148148148|
|  96|1017815.9166666666|
|  95|         1054325.6|
|  94|         1033536.2|
|  93|1026006.0606060605|
|  92| 970402.5476190476|
|  91| 1137372.731707317|
|  90|1062654.1621621621|
|  89|        1107839.15|
|  88|1031719.3548387097|
|  87|         1072285.2|
|  86|        1070444.25|
|  85|1056336.7435897435|
|  84|1117233.1290322582|
|  83|       1033965.925|
|  82|         1063498.0|
|  81|1053472.7878787878|
|  80| 991767.3783783783|
+----+------------------+
only showing top 20 rows

--- 0.5634124279022217 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales_view")

In [25]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached('home_sales_view')
print("Is home_sales_view cached?", is_cached)


Is home_sales_view cached? False
